In [32]:
import array, csv, fcntl, json, math, multiprocessing, os, random, re, shutil
import shapely, shapely.wkb, struct, subprocess, sys, threading, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [57]:
!ogr2ogr ac_2000.shp PG:"dbname=timelapse" -sql "SELECT * FROM tl_2000_tabblock WHERE stfid LIKE '42003%'"

In [61]:
!ogr2ogr ac_2010.shp PG:"dbname=timelapse" -sql "SELECT * FROM tl_2010_tabblock10 WHERE geoid10 LIKE '42003%'"

In [62]:
!file ac_2000.shp ac_2010.shp

ac_2000.shp: ESRI Shapefile version 1000 length 3221220 type Polygon
ac_2010.shp: ESRI Shapefile version 1000 length 10032082 type Polygon


In [63]:
!ls -l ac_20?0.*

-rw-rw-r-- 1 rsargent rsargent  1845765 Feb 14 20:23 ac_2000.dbf
-rw-rw-r-- 1 rsargent rsargent      143 Feb 14 20:23 ac_2000.prj
-rw-rw-r-- 1 rsargent rsargent  6442440 Feb 14 20:23 ac_2000.shp
-rw-rw-r-- 1 rsargent rsargent   194364 Feb 14 20:23 ac_2000.shx
-rw-rw-r-- 1 rsargent rsargent  4120610 Feb 14 20:37 ac_2010.dbf
-rw-rw-r-- 1 rsargent rsargent      143 Feb 14 20:35 ac_2010.prj
-rw-rw-r-- 1 rsargent rsargent 20064164 Feb 14 20:37 ac_2010.shp
-rw-rw-r-- 1 rsargent rsargent   244252 Feb 14 20:37 ac_2010.shx


In [66]:
!ls capture/tabblock_2010/tl_2010_42_tabblock10

tl_2010_42_tabblock10.dbf  tl_2010_42_tabblock10.shp.xml
tl_2010_42_tabblock10.prj  tl_2010_42_tabblock10.shx
tl_2010_42_tabblock10.shp


Compute all intersections between 2000 and 2010 block geometries
----------------------------------------------------------------

Actually, for now, just state 42 (PA), FIPS county 003 (Allegheny)

In [ ]:
SELECT
  c2000.bkgpidfp00 AS fips2000,
  c2010.geoid10 AS fips2010, 
  ST_Area(ST_Intersection(c2000.geom, c2010.geom)) / ST_Area(c2000.geom) AS weight_2000,
  ST_Area(ST_Intersection(c2000.geom, c2010.geom)) / ST_Area(c2010.geom) AS weight_2010,
  (ST_Dump(ST_Intersection(c2000.geom, c2010.geom))).geom AS geom
  INTO xwalk
  FROM census2000_block_groups c2000, census2010_block_groups c2010
 WHERE ST_Intersects(c2000.geom, c2010.geom) AND NOT ST_Touches(c2000.geom, c2010.geom);



In [47]:
query_psql('SELECT COUNT(c2000.stfid) '
           'FROM tl_2000_tabblock AS c2000, tl_2010_tabblock10 AS c2010 '
           'WHERE c2000.stfid LIKE \'42003%\''
           'AND c2010.geoid10 LIKE \'42003%\' '
           'AND ST_Intersects(c2000.geom, c2010.geom) '
           'AND NOT ST_Touches(c2000.geom, c2010.geom) ')

Execution of SELECT COUNT(c2000.stfid) FROM tl_2000_tabblock AS c2000, tl_2010_tabblock10 AS c2010 WHERE c2000.stfid LIKE '42003%'AND c2010.geoid10 LIKE '42003%' AND ST_Intersects(c2000.geom, c2010.geom) AND NOT ST_Touches(c2000.geom, c2010.geom)
took 22.3612 seconds and returned 1 rows


[(123117L,)]

In [46]:
query_psql('SELECT c2000.stfid, '
           'ST_Area(ST_Intersection(c2000.geom, c2010.geom)) / ST_Area(c2000.geom) AS weight_2000, '
           'ST_Area(ST_Intersection(c2000.geom, c2010.geom)) / ST_Area(c2010.geom) AS weight_2010 '
           'FROM tl_2000_tabblock AS c2000, tl_2010_tabblock10 AS c2010 '
           'WHERE c2000.stfid LIKE \'42003%\''
           'AND c2010.geoid10 LIKE \'42003%\' '
           'AND ST_Intersects(c2000.geom, c2010.geom) '
           'AND NOT ST_Touches(c2000.geom, c2010.geom) ')

Execution of SELECT c2000.stfid, ST_Area(ST_Intersection(c2000.geom, c2010.geom)) / ST_Area(c2000.geom) AS weight_2000, ST_Area(ST_Intersection(c2000.geom, c2010.geom)) / ST_Area(c2010.geom) AS weight_2010 FROM tl_2000_tabblock AS c2000, tl_2010_tabblock10 AS c2010 WHERE c2000.stfid LIKE '42003%'AND c2010.geoid10 LIKE '42003%' AND ST_Intersects(c2000.geom, c2010.geom) AND NOT ST_Touches(c2000.geom, c2010.geom) LIMIT 100000
took 117.154 seconds and returned 100000 rows


[('420031302001015', 0.0121001828051337, 0.0103836486813274),
 ('420031302001015', 0.721854861941446, 0.564294033234704),
 ('420031302001015', 0.0962308503982678, 0.0833674826774987),
 ('420031302001015', 0.169814104864544, 0.174158375183835),
 ('420031301002005', 0.0297312420881618, 0.315666101543513),
 ('420031301002005', 0.00737853748588418, 0.0236587349028963),
 ('420031301002005', 0.753530226529148, 0.745767997956525),
 ('420031301002005', 0.0269452041867945, 0.0908041769472226),
 ('420031301002005', 0.182414789709663, 0.326334189961214),
 ('420031301002006', 0.40449857606038, 0.369094671848271),
 ('420031301002006', 0.332424140311418, 0.325679173198031),
 ('420031301002006', 0.263077283629498, 0.0240253074016802),
 ('420031301002007', 0.350069728111789, 0.29342963673436),
 ('420031301002007', 0.374468888937857, 0.123073604107655),
 ('420031301002007', 0.231823926073482, 0.0194478905061239),
 ('420031301002007', 0.0436374568767006, 0.0165480967732384),
 ('420031301002008', 1.42590

In [45]:
query_psql('SELECT COUNT(*) FROM tl_2000_tabblock AS c2000 WHERE c2000.stfid LIKE \'42003%\'')

Execution of SELECT COUNT(*) FROM tl_2000_tabblock AS c2000 WHERE c2000.stfid LIKE '42003%'
took 3.24341 seconds and returned 1 rows


[(24283L,)]

In [20]:
max_gid_query = 'SELECT gid FROM od_{jt}_{year} ORDER BY gid DESC LIMIT 1'.format(jt=jt, year=year)
max_gid = query_psql(max_gid_query)[0][0]
print 'Maximum gid in od_{jt}_{year} is {max_gid}'.format(jt=jt, year=year, max_gid=max_gid)

Execution of SELECT gid FROM od_JT01_2011 ORDER BY gid DESC LIMIT 1
took 0.00324702 seconds and returned 1 rows
Maximum gid in od_JT01_2011 is 106605364


In [30]:
max_psql_parallelism = 8
psql_semaphore = multiprocessing.Semaphore(max_psql_parallelism)

def query_psql_throttled(query, quiet=False):
    psql_semaphore.acquire()
    try:
        rows = query_psql(query, quiet=quiet)
    finally:
        psql_semaphore.release()
    return rows


In [22]:
def write_points(points):
    fcntl.flock(point_output, fcntl.LOCK_EX)
    try:
        point_output.write(''.join(points))
        point_output.flush()
    finally:
        fcntl.flock(point_output, fcntl.LOCK_UN)

def process_employment_shard(gid_start, gid_end):
    query = (
            "SELECT w.geom, h.geom, od.se01, od.se02, od.se03 "
            "FROM od_{jt}_{year} od  "
            "JOIN tl_2010_tabblock10 w ON od.w_geocode = w.geoid10 "
            "JOIN tl_2010_tabblock10 h ON od.h_geocode = h.geoid10 "
            "WHERE od.gid BETWEEN {gid_start} AND {gid_end} "
            #"AND od.h_geocode LIKE '42%' "
            ).format(jt=jt, year=year, gid_start=gid_start, gid_end=gid_end)
    rows = query_psql_throttled(query, quiet=True)
    begin_time = time.time()
    if len(rows) == 0:
        sys.stdout.write("No rows, aborting shard %d\n" % gid_start)
        return []
    sys.stdout.write("Starting shard %d with %d rows\n" % (gid_start, len(rows)))

    points = []
    wgs84_geod = pyproj.Geod(ellps='WGS84')
    for (workGeom, homeGeom, se01, se02, se03) in rows:
        workPolygon = shapely.wkb.loads(workGeom, hex=True)
        homePolygon = shapely.wkb.loads(homeGeom, hex=True)

        # Compute distance on the sphere
        dist = wgs84_geod.inv(workPolygon.centroid.x, workPolygon.centroid.y,
                              homePolygon.centroid.x, homePolygon.centroid.y)[2]
        
        for (count, color) in [(se01, se01_color), (se02, se02_color), (se03, se03_color)]:
            pcolor = pack_color(color)
            for i in range(count):
                wpoint = LonLatToPixelXY(randomPoint(workPolygon))
                hpoint = LonLatToPixelXY(randomPoint(homePolygon))
                points.append(struct.pack('<ffffff', wpoint[0], wpoint[1], hpoint[0], hpoint[1], dist, pcolor))
        
        if len(points) >= 10000:
            write_points(points)
            points = []
    write_points(points)
    sys.stdout.write("Finished shard %d with %d rows in %g seconds\n" % (gid_start, len(rows), time.time() - begin_time))


In [23]:
dest = 'od-{jt}-{year}.bin'.format(jt=jt, year=year)
print "Opening binary file %s for appending data" % dest

open(dest, 'wb') # truncate the file
point_output = open(dest, 'ab')

shard_size = 50000
pool = multiprocessing.Pool()

results = []

print "Starting shards with maximum index %d" % max_gid

for gid_start in range(0, max_gid + 1, shard_size):
    results.append(pool.apply_async(process_employment_shard, (gid_start, gid_start + shard_size - 1)))

print "No more shards to create.  Waiting for shards to finish."

pool.close()
pool.join()

for res in results:
    res.get()

Opening binary file od-JT01-2011.bin for appending data
Starting shard 0 with 49999 rows
Starting shard 200000 with 50000 rows
Starting shard 150000 with 50000 rows
Starting shard 50000 with 50000 rows
Starting shard 250000 with 50000 rows
Starting shard 100000 with 50000 rows
Starting shard 300000 with 50000 rows
Starting shard 350000 with 50000 rows
Starting shard 400000 with 50000 rows
Starting shard 450000 with 50000 rows
Starting shard 500000 with 50000 rows
Starting shard 550000 with 50000 rows
Starting shard 600000 with 50000 rows
Starting shard 650000 with 50000 rows
Starting shard 700000 with 50000 rows
Starting shard 750000 with 50000 rows
Starting shard 800000 with 50000 rows
Starting shard 950000 with 50000 rows
Starting shard 900000 with 50000 rows
Starting shard 850000 with 50000 rows
Starting shard 1000000 with 50000 rows
Starting shard 1050000 with 50000 rows
Starting shard 1100000 with 50000 rows
Starting shard 1150000 with 50000 rows
Starting shard 1200000 with 50000 

In [27]:
record_len = 6 * 4
print '%s has %.1f records (points)' % (dest, os.stat(dest).st_size / float(record_len))

od-JT01-2011.bin has 120322051.0 records (points)


Compute dot home location for "non-employed" persons
----------------------------------------------------

Compute non-employed persons by subtracting LODES primary jobs from "working age population" (15-64) from 2010 Decennial census

In [31]:
def get_geocode_at(idx):
    q = "SELECT geocode FROM working_age_blocks ORDER BY geocode OFFSET {idx} LIMIT 1 ".format(idx=idx)
    return query_psql(q, quiet=True)[0][0]

def process_nonemployment_shard(shard_start, shard_end):
    query = (
            "SELECT MIN(h.geom), MIN(t.working_age_pop), "
            "SUM(od.se01 + od.se02 + od.se03)"
            "FROM working_age_blocks t "
            "JOIN tl_2010_tabblock10 h ON t.geocode = h.geoid10 "
            "LEFT JOIN od_{jt}_{year} od ON t.geocode = od.h_geocode "
            "WHERE t.geocode BETWEEN '{geocode_start}' AND '{geocode_end}' "
            #"AND t.geocode LIKE '42%' "
            "GROUP BY t.geocode "
            ).format(jt=jt, year=year, geocode_start=get_geocode_at(shard_start), geocode_end=get_geocode_at(shard_end))
    rows = query_psql_throttled(query, quiet=True)
    begin_time = time.time()
    if len(rows) == 0:
        sys.stdout.write("No rows, aborting nonemployment shard %d\n" % shard_start)
        return []
    sys.stdout.write("Starting nonemployment shard %d with %d rows\n" % (shard_start, len(rows)))

    points = []
    pcolor = pack_color(nonemployed_color)
    for (homeGeom, working_age_pop, employed_pop) in rows:
        if employed_pop == None:
            # LODES records no employment for this home block
            employed_pop = 0
        homePolygon = shapely.wkb.loads(homeGeom, hex=True)

        dist = 0
        count = max(working_age_pop - employed_pop, 0)
        # TODO:  look at negative values;  maybe show them as a different color until we better understand
        for i in range(count):
            hpoint = LonLatToPixelXY(randomPoint(homePolygon))
            points.append(struct.pack('<ffffff', hpoint[0], hpoint[1], hpoint[0], hpoint[1], dist, pcolor))
        
        if len(points) >= 10000:
            write_points(points)
            points = []
    write_points(points)
    sys.stdout.write("Finished nonemployment shard %d with %d rows in %g seconds\n" % (shard_start, len(rows), time.time() - begin_time))


In [ ]:
max_shard_idx = query_psql("SELECT COUNT(*) FROM working_age_blocks", quiet=True)[0][0] - 1

shard_size = 50000

pool = multiprocessing.Pool()


print "Starting shards with maximum index %d" % max_shard_idx

results = []
for shard_start in range(0, max_shard_idx + 1, shard_size):
    results.append(pool.apply_async(process_nonemployment_shard, (shard_start, min(max_shard_idx, shard_start + shard_size - 1))))

print "No more shards to create.  Waiting for shards to finish."

pool.close()
pool.join()

for res in results:
    res.get()

Starting shards with maximum index 11155485
No more shards to create.  Waiting for shards to finish.
Starting nonemployment shard 300000 with 50000 rows
Starting nonemployment shard 100000 with 50000 rows
Starting nonemployment shard 150000 with 50000 rows
Starting nonemployment shard 200000 with 50000 rows
Starting nonemployment shard 450000 with 50000 rows
Starting nonemployment shard 400000 with 50000 rows
Starting nonemployment shard 550000 with 50000 rows
Starting nonemployment shard 600000 with 50000 rows
Finished nonemployment shard 300000 with 50000 rows in 44.5443 seconds
Starting nonemployment shard 50000 with 50000 rows
Starting nonemployment shard 650000 with 50000 rows
Starting nonemployment shard 0 with 50000 rows
Starting nonemployment shard 500000 with 50000 rows
Starting nonemployment shard 250000 with 50000 rows
Starting nonemployment shard 350000 with 50000 rows
Starting nonemployment shard 850000 with 50000 rows
Finished nonemployment shard 600000 with 50000 rows in

In [33]:
print '%s has %.1f records (points)' % (dest, os.stat(dest).st_size / float(record_len))

od-JT01-2011.bin has 223234100.0 records (points)


Randomize point order and write them to binary file for web client
------------------------------------------------------------------

In [36]:
points = []

npoints = os.stat(dest).st_size / record_len

points_file = open(dest, 'rb')
points = [points_file.read(record_len) for i in range(npoints)]
points_file.close()
print "Randomizing %d points..." % len(points)
random.shuffle(points)        

print "Writing data back to binary file %s" % dest
points_file = open(dest, 'wb')
for b in points:
    points_file.write(b)
points_file.close()
print '%s has %.1f records (points)' % (dest, os.stat(dest).st_size / float(record_len))

Randomizing 223234100 points...
Writing data back to binary file od-JT01-2011.bin
od-JT01-2011.bin has 223234100.0 records (points)


Look at some block correspondences in the 2000 to 2010 block crosswalk
-----------------------------------------------------------------------

In [67]:
!ls -l  ~/Downloads/TAB*

ls: cannot access '/home/rsargent/Downloads/TAB*': No such file or directory


In [68]:
!mv ~/TAB2000* .

In [70]:
!unzip TAB2000*zip

Archive:  TAB2000_TAB2010_ST_01_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_02_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_04_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_05_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_06_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_08_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_09_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_10_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_11_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_12_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_13_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_15_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_16_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_17_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_18_v2.zip
caution: filename not matched:  TAB2000_TAB2010_ST_19_v2.zip
caution: filename not matched:  TAB2000_TAB201

In [71]:
!unzip TAB2000_TAB2010_ST_02_v2.zip

Archive:  TAB2000_TAB2010_ST_02_v2.zip
replace TAB2000_TAB2010_ST_02_v2.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [72]:
!ls -l TAB2000*ST_42*

-rw-rw-r-- 1 rsargent rsargent 127370376 Feb 25  2011 TAB2000_TAB2010_ST_42_v2.txt
-rw-r--r-- 1 rsargent rsargent   7040655 Feb 15 13:43 TAB2000_TAB2010_ST_42_v2.zip


In [77]:
!head -1 TAB2000_TAB2010_ST_42_v2.txt 

In [81]:
!grep ^42,003,0409..,3000 TAB2000_TAB2010_ST_42_v2.txt

In [82]:
!grep ^42,003,0409..,3001 TAB2000_TAB2010_ST_42_v2.txt

In [83]:
!grep ^42,003,0409..,3002 TAB2000_TAB2010_ST_42_v2.txt

In [84]:
!grep ,42,003,040900,2001, TAB2000_TAB2010_ST_42_v2.txt